In [30]:
from random import randint

def findRoot(f, p:int):
    R.<x> = PolynomialRing(GF(p))
    myRoot = []
    g = gcd(f, x^p-x)
    n = g.degree()
    if n == 0:
        print('Корней нет')
        return []
    elif n == 1:
        return -g[0]
    elif n == p:
        print('Корни - все элементы Fp')
        return []
    
    stack = []
    stack.append(g)
    while(stack):
        d = 1
        while(d == R(1) or d == stack[0]):
            d = gcd(R((x+randint(1, p))**((p-1)//2) - 1), R(stack[0]))
        
        n = d.degree()
        if(n == 1):
            myRoot.append(-d[0])
            tmp = R(stack[0] / d)
            if tmp.degree() == 1:
                myRoot.append(-tmp[0])
                stack.pop(0)
            else:
                stack.pop(0)
                stack.append(tmp)
            
        elif(n == stack[0].degree() - 1):
            q = R(stack[0] / d)
            myRoot.append(-q[0])
            stack.pop(0)
            stack.append(d)
        
        elif(n >=2 and n < stack[0].degree() - 1):
            stack.append(d)
            stack.append(stack[0]/d)
            stack.pop(0)
    
    return myRoot

p = 2128747
R.<x> = PolynomialRing(GF(p))
# f = x**18 + 1576544*x**17 + 1269679*x**16 + 421388*x**15 + \
#   + 420073*x**14 + 93274*x**13 + 419461*x**12 + 1552845*x**11 + \
#   + 268691*x**10 + 1564101*x**9 + 593897*x**8 + 1360879*x**7 + \
#   + 2039035*x**6 + 779162*x**5 + 857188*x**4 + 1791605*x**3 + \
#   + 1708777*x**2 + 27962*x + 1688424
#f = (x**2 - 9)*(x**5 + 4*x**2 + x + 5616)
f = x - 3
print(f)
print(f"myRoots = {findRoot(f, p)}")
print(f"{f.roots() = }")
        
        


x + 2128744
myRoots = 3
f.roots() = [(3, 1)]


In [7]:
#P = Primes()
#q = P.next(2**5)

#Берлекэмп

def berlekamp(f, degf):
    #create matrix
    R.<x> = PolynomialRing(GF(q))
    B = []
    znach = 1
    i=1
    B.append([1] + [0]*(degf-1))
    while(i<degf):
        znach = (znach * x**q) % f
        cf = []
        for j in range(degf):
            cf.append(znach[j])
        B.append(cf)   
        i+=1

   
    #(B-E)
    for i in range(degf):
        B[i][i] -=1

    
    #(B-E).transpose.rank
    m = Matrix(GF(q), B).T
    if m.rank() == degf - 1:
        return []

    e = m.right_kernel().basis_matrix().rows()
    r = len(e)
    l = 0
    i = 1
    U = [0] * r
    U[0] = f
    while i < r:
        h = sum([e[i][j] * R(x**j) for j in range(int(degf))])
        j = 0
        while j <= l:
            g = set()
            for c in range(q):
                gCur = gcd(U[j], h - R(c))
                g.add(gCur)
            g.discard(R(1)) 
            g = list(g)
            U[j] = g[0]
            t = 1
            while t < len(g):
                U[l+t] = g[t]
                t+=1    
            l = l + len(g) - 1
            if l == r - 1: 
                return U
            j+=1
            
        i = i + 1
    return U
                
    
    
#q = 1922603
q = 3
gf = GF(q)
R.<x> = PolynomialRing(gf)

# f_orig = x**18 + 7288*x**17 + 1569197*x**16 + 462295*x**15 +\
#     + 832234*x**14 + 72126*x**13 + 410429*x**12 + 1874129*x**11 +\
#     + 531482*x**10 + 1741564*x**9 + 1632964*x**8 + 1066504*x**7 +\
#     + 1105905*x**6 + 1062657*x**5 + 417643*x**4 + 359234*x**3 +\
#     + 131984*x**2 + 220096*x + 66760

#f_orig = x**2 * (x+3) ** 2 * (x+4) ** 3
#f_orig = (x+3)**54 * x**101 * (x+1)**135
#f_orig = R(x**5 * (x+3)**10 * (x+4)**13)
#print(f"f = {f_orig}")
f_orig = R(x**33 + x**24 + x**15 + x**21 + x**30 + x**12 + x**18 + x**9 +1)
f_tmp = f_orig
result = []


while f_tmp != R(1):
    f_ = f_tmp.derivative()
    g = R(gcd(f_tmp, f_))
    if g != f_tmp:
        f = R(f_tmp / g)
    else:
        f = f_tmp
        while(g == f):
            f = R(f.list()[0::q])
            f_ = f.derivative()
            g = R(gcd(f, f_))
        f = R(f / g)
    print(f)
    roots = berlekamp(f, f.degree())
    if roots:
        i, k = 0, 0
        while i < len(roots):
            if(f_tmp % roots[i] == 0):
                f_tmp = R(f_tmp / roots[i])
                k+=1
            else:
                result.append([roots[i], k])
                k, i = 0, i+1
    else:
        k=0
        while f_tmp % f == 0:
            f_tmp = f_tmp // f
            k+=1
        result.append([f, k])
    
print(f_orig)
print(f"my factor = {result}")
print("sage:")
print(f_orig.factor())





x^4 + 2*x^3 + 2*x^2 + 2*x + 2
x^33 + x^30 + x^24 + x^21 + x^18 + x^15 + x^12 + x^9 + 1
my factor = [[x + 2, 24], [x^3 + 2*x + 1, 3]]
sage:
(x + 2)^24 * (x^3 + 2*x + 1)^3
